Used Random forest solved the car price prediction regression problem

In [36]:
#  Import liberies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [37]:
#  Load  dataset
df=pd.read_csv("cardekho_imputated.csv")
df.head()

,Unnamed: 0,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


Data Cleaning

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15411 entries, 0 to 15410
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         15411 non-null  int64  
 1   car_name           15411 non-null  object 
 2   brand              15411 non-null  object 
 3   model              15411 non-null  object 
 4   vehicle_age        15411 non-null  int64  
 5   km_driven          15411 non-null  int64  
 6   seller_type        15411 non-null  object 
 7   fuel_type          15411 non-null  object 
 8   transmission_type  15411 non-null  object 
 9   mileage            15411 non-null  float64
 10  engine             15411 non-null  int64  
 11  max_power          15411 non-null  float64
 12  seats              15411 non-null  int64  
 13  selling_price      15411 non-null  int64  
dtypes: float64(2), int64(6), object(6)
memory usage: 1.6+ MB


In [39]:
df.isna().sum()

Unnamed: 0           0
car_name             0
brand                0
model                0
vehicle_age          0
km_driven            0
seller_type          0
fuel_type            0
transmission_type    0
mileage              0
engine               0
max_power            0
seats                0
selling_price        0
dtype: int64

In [40]:
df.describe()
#  That help to their are oulier present aur not

,Unnamed: 0,vehicle_age,km_driven,mileage,engine,max_power,seats,selling_price
count,15411.000000,15411.000000,1.541100e+04,15411.000000,15411.000000,15411.000000,15411.000000,1.541100e+04
mean,9811.857699,6.036338,5.561648e+04,19.701151,1486.057751,100.588254,5.325482,7.749711e+05
std,5643.418542,3.013291,5.161855e+04,4.171265,521.106696,42.972979,0.807628,8.941284e+05
min,0.000000,0.000000,1.000000e+02,4.000000,793.000000,38.400000,0.000000,4.000000e+04
25%,4906.500000,4.000000,3.000000e+04,17.000000,1197.000000,74.000000,5.000000,3.850000e+05
50%,9872.000000,6.000000,5.000000e+04,19.670000,1248.000000,88.500000,5.000000,5.560000e+05
75%,14668.500000,8.000000,7.000000e+04,22.700000,1582.000000,117.300000,5.000000,8.250000e+05
max,19543.000000,29.000000,3.800000e+06,33.540000,6592.000000,626.000000,9.000000,3.950000e+07


In [41]:
df.head(2)

,Unnamed: 0,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.7,796,46.3,5,120000
1,1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.9,1197,82.0,5,550000


In [42]:
df.drop("car_name",axis=1,inplace=True)
df.drop("brand",axis=1,inplace=True)

In [43]:
df.head()

,Unnamed: 0,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [44]:
#  Getting all different types of features
# Numerical feature
numerical_feature=[feature for feature in df.columns if df[feature].dtype!="object"]
print("numerical_feature",len(numerical_feature))
# categorical feature
categorical_feautre=[feature for feature in df.columns if df[feature].dtype=="object"]
print("categorical_feautre",len(categorical_feautre))
# Descrite feature
Descrite_feature=[feature for feature in numerical_feature if len(df[feature].unique())<=25]
print("Descrite_feature",len(Descrite_feature))
# Continuous feature
Continuous_feature=[feature for feature in numerical_feature if feature not in Descrite_feature]
print("Continuous_feature",len(Continuous_feature))

numerical_feature 8
categorical_feautre 4
Descrite_feature 2
Continuous_feature 6


In [45]:
#  Split Independent feature and Dependent feature
X=df.drop("selling_price",axis=1)
y=df["selling_price"]

In [46]:
X.head()

,Unnamed: 0,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
0,0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5
1,1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5
2,2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5
3,3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5
4,4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5


### Feature Enginerring

#### One hot encoding for coloumns which had lesser unique calues and not oridal
1. one hot encoding is a process by which categorical variable are converted into a form that could be provided to ml algorithm to do a better job in prodiction

In [47]:
X["model"]

0            Alto
1           Grand
2             i20
3            Alto
4        Ecosport
           ...   
15406         i10
15407      Ertiga
15408       Rapid
15409      XUV500
15410        City
Name: model, Length: 15411, dtype: object

In [51]:
from sklearn.preprocessing import LabelEncoder
Label_Encoder=LabelEncoder()
X["model"]=Label_Encoder.fit_transform(X["model"])

In [52]:
#  Create columns Transformer with 3 type of transformer

num_feature=X.select_dtypes(exclude="object").columns
onehot_columns=["seller_type","fuel_type","transmission_type"]


#  Import libereies
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
Standard_Scaler=StandardScaler()
One_HotEncoder=OneHotEncoder(drop="first")


preprocessor=ColumnTransformer(

    [
        ("OneHotEncoder",One_HotEncoder,onehot_columns),
        ("StandardScaler",Standard_Scaler,num_feature)

    ],remainder='passthrough' # it keep another feature as it remaining same
)


In [53]:
# Split the data
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=42)


In [ ]:
#  Tranform the dataset
X_train=preprocessor.fit_transform(X_train)
X_test=preprocessor.transform(X_test)

In [ ]:
# Convert into dataframe
X_train=pd.DataFrame(X_train)
X_test=pd.DataFrame(X_test)

In [63]:
X_train.shape,y_train.shape,X_test.shape

((12328, 15), (12328,), (3083, 15))

### Model Training And Model Selection

In [64]:
#  import all regreesion libearay
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score

In [83]:
# Create the function to evaluate model
def evlation(true,predicted):
    mes=mean_squared_error(true,predicted)
    print("mean_squared_error",mes)
    mae=mean_absolute_error(true,predicted)
    print("mean_absolute_error",mae)
    rmse=np.sqrt(mes)
    print("Root mean square",rmse)
    Rr2_score=r2_score(true,predicted)
    print("r2_score",Rr2_score)
    return 0
    

In [74]:
# train the model:
# Beginning model training
Model={
    "Linear_Regression":LinearRegression(),
    "Lasso_regression":Lasso(),
    "Ridge_regression":Ridge(),
    "KNN":KNeighborsRegressor(),
    "Tree_regression":DecisionTreeRegressor(),
    "Random_ForestRegressor":RandomForestRegressor()
}

In [84]:
for i in range(len(list(Model))):
    Models=list(Model.values())[i]
    Models.fit(X_train,y_train)

    # make pedictions
    Y_train_prediction= Models.predict(X_train)
    y_test_prediction=Models.predict(X_test)

    # Evaluate Train test dataset
    print(list(Model.keys())[i])
    print("Training model evaluation")
    print(evlation(y_train,Y_train_prediction))

    print("\n")
    
    print("Tesing model evaltion")
    print(evlation(y_test,y_test_prediction))
    print("___________________________________________________________________________________________________")



KNN
Training model evaluation
mean_squared_error 0.0
mean_absolute_error 0.0
Root mean square 0.0
r2_score 1.0
0


Tesing model evaltion
mean_squared_error 91100314407.62785
mean_absolute_error 114091.08455938465
Root mean square 301828.2862947538
r2_score 0.8789817772436409
0
___________________________________________________________________________________________________
Tree_regression
Training model evaluation
mean_squared_error 288642592229.072
mean_absolute_error 207867.53731343284
Root mean square 537254.680974556
r2_score 0.6441057803974807
0


Tesing model evaltion
mean_squared_error 185294114422.84302
mean_absolute_error 219276.39474537788
Root mean square 430458.02864256466
r2_score 0.753854149017203
0
___________________________________________________________________________________________________
Random_ForestRegressor
Training model evaluation
mean_squared_error 156347420564.54486
mean_absolute_error 170890.56380794372
Root mean square 395407.9166690329
r2_score 0.807

Hyperparameter tuning


In [90]:
#  Create the paramter kNN
weights=['uniform', 'distance']
algorithm=['ball_tree', 'kd_tree', 'brute']
p=[1,2]
n_neighbors=[1,2,3,4,5,6,7,8,9,10]

Knn_param=[{

    'n_neighbors':n_neighbors,'weights':weights,'algorithm':algorithm,'p':p
}]

In [88]:
#  Decision tree hyperparameter tuning
criterion=['squared_error', 'friedman_mse', 'absolute_error', 'poisson']
splitter=['best', 'random']
max_depth=[1,2,3,4,5,10,15]
max_features=['auto', 'sqrt', 'log2']
decisiontree_param1=[{
    'criterion':criterion,'splitter':splitter,'max_depth':max_depth,'max_features':max_features
}]

In [89]:
# Random forest hyperparamter tununig
criterion=['squared_error', 'absolute_error', 'friedman_mse', 'poisson']
max_features=['sqrt', 'log2']
max_depth=[1,2,3,4,5,7,10,15]
n_estimators=[50, 100, 200, 300, 500]

random_param=[{
    "criterion":criterion,"max_features":max_features,'max_depth':max_depth,'n_estimators':n_estimators

}]



In [91]:
#Model list for hyperparameter tuning
Randomcv_model=[
    ("KNN",KNeighborsRegressor(),Knn_param),
    ("decisiontree",DecisionTreeRegressor(),decisiontree_param1),
    ("randomforest",RandomForestRegressor(),random_param)
] 

In [97]:
#  Import libearies
from sklearn.model_selection import RandomizedSearchCV
model_param={}

for name,model,param in Randomcv_model:
    random=RandomizedSearchCV(estimator=model,
                              param_distributions=param,
                              cv=5,
                              verbose=2
                              ,n_jobs=-1)
    random.fit(X_train,y_train)
    model_param[name]=random.best_params_

for name in model_param :
    print(f"-------------------Best param for {name}---------------------------")
    print(model_param[name])   

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits
-------------------Best param for KNN---------------------------
{'weights': 'uniform', 'p': 1, 'n_neighbors': 2, 'algorithm': 'brute'}
-------------------Best param for decisiontree---------------------------
{'splitter': 'best', 'max_features': 'log2', 'max_depth': 4, 'criterion': 'friedman_mse'}
-------------------Best param for randomforest---------------------------
{'n_estimators': 200, 'max_features': 'log2', 'max_depth': 15, 'criterion': 'poisson'}


In [102]:
# Retrain the model:
Model={
    "KNN":KNeighborsRegressor(weights="uniform",p=1,n_neighbors=2,algorithm="brute"),
    "Tree_regression":DecisionTreeRegressor(splitter="best",max_features="log2",max_depth=10,criterion="friedman_mse"),
    "Random_ForestRegressor":RandomForestRegressor(n_estimators=200,max_features="log2",max_depth=15,criterion="poisson")
}


In [103]:
for name, model in Model.items():
    model.fit(X_train, y_train)

    # Predictions
    re_y_train_pred = model.predict(X_train)
    re_y_test_pred = model.predict(X_test)

    print(f"\nModel: {name}")
    print("Training Data Evaluation")
    print(evlation(re_y_train_pred, y_train))
    print("Testing Data Evaluation")
    print(evlation(re_y_test_pred, y_test))
    print("_" * 70)



Model: KNN
Training Data Evaluation
mean_squared_error 43127358594.25697
mean_absolute_error 65658.6226476314
Root mean square 207671.2753229415
r2_score 0.935923345976331
0
Testing Data Evaluation
mean_squared_error 108719034954.7924
mean_absolute_error 121375.64871878042
Root mean square 329725.6965339408
r2_score 0.8333756205022754
0
______________________________________________________________________

Model: Tree_regression
Training Data Evaluation
mean_squared_error 50431595152.00578
mean_absolute_error 127578.51298994162
Root mean square 224569.80017804215
r2_score 0.9336952621817277
0
Testing Data Evaluation
mean_squared_error 198572040603.1519
mean_absolute_error 171062.2154439615
Root mean square 445614.2284568031
r2_score 0.7750482379747761
0
______________________________________________________________________

Model: Random_ForestRegressor
Training Data Evaluation
mean_squared_error 20110851477.512733
mean_absolute_error 56893.64270508701
Root mean square 141812.7338341